In [0]:
# 引入使用的依赖库
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import GradientBoostingClassifier

In [19]:
# 读取数据，归一化
all_pd = pd.read_csv( "data_train.csv", sep=',', names=['f1', 'f2','f3','f4','f5','f6','label'])
X = np.array(all_pd.iloc[:,4:6].values.tolist(), dtype='float32')
y = np.array(all_pd.iloc[:,-1].values.tolist(), dtype='float32')

scaler = sklearn.preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)

print(X.shape)
print(y.shape)

(85500, 2)
(85500,)


In [0]:
# 随机划分train/val, 80:20
train_X,val_X, train_y, val_y = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [0]:
# 定义f1 metric
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
# 定义MLP模型结构
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(8, input_dim=2, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=[f1])

In [0]:
# 定义callback
from keras.callbacks import EarlyStopping,ModelCheckpoint
callbacks = [EarlyStopping(monitor='val_f1', min_delta=0.001, patience=10, verbose=0, mode='max'), 
    ModelCheckpoint("w.h5", monitor='val_f1', verbose=0, save_best_only=True, save_weights_only=True, mode='max', period=1)]

In [28]:
# 训练
model.fit(train_X, train_y,epochs=100,batch_size=2048,validation_data=(val_X, val_y), callbacks=callbacks)

Train on 68400 samples, validate on 17100 samples
Epoch 1/100
68400/68400 [==============================] - 1s 9us/step - loss: 0.6953 - f1: 0.2987 - val_loss: 0.6914 - val_f1: 0.5803
Epoch 2/100
68400/68400 [==============================] - 0s 4us/step - loss: 0.6928 - f1: 0.2910 - val_loss: 0.6892 - val_f1: 0.8367
Epoch 3/100
68400/68400 [==============================] - 0s 5us/step - loss: 0.6917 - f1: 0.3685 - val_loss: 0.6861 - val_f1: 0.7821
Epoch 4/100
68400/68400 [==============================] - 0s 4us/step - loss: 0.6886 - f1: 0.4984 - val_loss: 0.6823 - val_f1: 0.7746
Epoch 5/100
68400/68400 [==============================] - 0s 5us/step - loss: 0.6859 - f1: 0.4985 - val_loss: 0.6774 - val_f1: 0.7821
Epoch 6/100
68400/68400 [==============================] - 0s 5us/step - loss: 0.6841 - f1: 0.5037 - val_loss: 0.6721 - val_f1: 0.7937
Epoch 7/100
68400/68400 [==============================] - 0s 5us/step - loss: 0.6803 - f1: 0.5155 - val_loss: 0.6655 - val_f1: 0.8020
Epoch

In [0]:
# 产出预测结果
test_data = pd.read_csv( "data_test.csv", sep=',', names=['f1', 'f2','f3','f4','f5','f6']).as_matrix()
result = open("result.csv", "w+", encoding="UTF-8")
test_data = scaler.transform(test_data)

res = model.predict(test_data)

for i,r in enumerate(res):
    result.write("{},{}\n".format(i+1,int(round(r[0])) ))
result.close()